In [ ]:
import pandas as pd
from functools import reduce
import sys
from random import randint
from time import sleep
sys.path.append("../..")
from config.settings import get_data_url, OddsMatchColumn, ODDS_COLUMN_VALUES, get_data_directory_matches,DIRECTORY_MLS_MATCHES_ODDS, TEAM_NAME_TO_CLUB, DIRECTORY_COMBINED_MATCHES_CLEAN, extract_number, Club, RELEVANT_YEARS, DIRECTORY_COMBINED_MATCHES_CLEAN_AGG, LINEAR_COLS, POINTS_MAP, MatchLogTypes
pd.set_option('display.max_columns', 70)
df = pd.read_csv("../../data/Combined_Team_Stats.csv")
df_matches = pd.read_csv("../../data/Combined_MLS_Data2024.csv")
df_odds = pd.read_excel("../../data/mls_historical_odds.xlsx")

In [2]:
def parse_date_with_multiple_formats(date_str):
    for fmt in ('%m/%d/%Y', '%Y-%m-%d %H:%M:%S'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError as e:
            print(f"{date_str} , Error occurred: {e}")
            continue
    return pd.NaT  # Return Not a Time for unparseable formats


In [3]:
def normalize_team_names(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    for col in columns:
        df[col] = df[col].map(lambda x: TEAM_NAME_TO_CLUB.get(x, None).value if x in TEAM_NAME_TO_CLUB else x)
    return df


In [4]:

filtered_df = df_odds[~df_odds['Season'].between(2012, 2017)]
filtered_df['Date'] = filtered_df['Date'].apply(parse_date_with_multiple_formats)

C:\Users\jamal\AppData\Local\Temp\ipykernel_24256\811614169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = filtered_df['Date'].apply(parse_date_with_multiple_formats)


In [ ]:
df_odds_clean = filtered_df[ODDS_COLUMN_VALUES]
df_odds_clean = normalize_team_names(filtered_df, [OddsMatchColumn.HOME_TEAM.value, OddsMatchColumn.AWAY_TEAM.value])
print(df_odds_clean[OddsMatchColumn.HOME_TEAM.value].unique())
df_odds_clean.to_csv(DIRECTORY_MLS_MATCHES_ODDS)

['Nashville-SC' 'Atlanta-United' 'Charlotte-FC' 'DC-United'
 'FC-Cincinnati' 'Inter-Miami' 'Orlando-City' 'Philadelphia-Union'
 'Austin-FC' 'FC-Dallas' 'Vancouver-Whitecaps-FC' 'Seattle-Sounders'
 'Portland-Timbers' 'Columbus-Crew' 'New-England-Revolution'
 'New-York-Red-Bulls' 'Chicago-Fire' 'St-Louis-City' 'Colorado-Rapids'
 'San-Jose-Earthquakes' 'Los-Angeles-FC' 'LA-Galaxy' 'Houston-Dynamo'
 'New-York-City-FC' 'Toronto-FC' 'Minnesota-United' 'Real-Salt-Lake'
 'Sporting-Kansas-City' 'CF-Montreal']


C:\Users\jamal\AppData\Local\Temp\ipykernel_24256\242616791.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(lambda x: TEAM_NAME_TO_CLUB.get(x, None).value if x in TEAM_NAME_TO_CLUB else x)


In [11]:
def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [
        col[1].strip() if isinstance(col, tuple) and len(col) > 1 else str(col).strip()
        for col in df.columns
    ]
    return df

def normalize_date_column(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if "date" in col.lower():
            df.rename(columns={col: "Date"}, inplace=True)
            df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
            break
    return df

In [ ]:
def load_fbref_table(url: str, ATTRs: dict = {"id": "matchlogs_for"}) -> pd.DataFrame:
    try:
        tables = pd.read_html(url, ATTRs=ATTRs)
        df = tables[0]
        df = flatten_columns(df)
        df = normalize_date_column(df)
        return df
    except Exception as e:
        print(f"Error loading table from {url}: {e}")
        return pd.DataFrame()

In [13]:
def merge_fbref_matchlogs(club: Club, year: int, log_types: list[MatchLogTypes]) -> pd.DataFrame:
    dfs = []
    for log_type in log_types:
        sleep(randint(15,20))
        url = get_data_url(club, log_type.value, year)
        df = load_fbref_table(url)
        if not df.empty:
            dfs.append(df)
    if not dfs:
        return pd.DataFrame()

    return reduce(lambda left, right: pd.merge(left, right, on="Date", how="outer"), dfs)

In [ ]:
for year in RELEVANT_YEARS:
    df_list = []
    for c in Club:
        merged_df = merge_fbref_matchlogs(
            c,
            year,
            [MatchLogTypes.Shooting, MatchLogTypes.GoalAndShotCreation]
        )
        merged_df['Team'] = c.value
        if not merged_df.empty:
            merged_df['Team'] = c.value
            df_list.append(merged_df)

    combined_df = pd.concat(df_list, ignore_index=True)
    combined_df.to_csv(get_data_directory_matches(year))



Error loading table from https://fbref.com/en/squads/eb57545a/2021/matchlogs/all_comps/shooting/Charlotte-FC-Match-Logs-All-Competitions: HTTP Error 500: Internal Server Error
Error loading table from https://fbref.com/en/squads/eb57545a/2021/matchlogs/all_comps/gca/Charlotte-FC-Match-Logs-All-Competitions: HTTP Error 500: Internal Server Error
Error loading table from https://fbref.com/en/squads/91b092e1/2021/matchlogs/all_comps/shooting/San-Diego-FC-Match-Logs-All-Competitions: HTTP Error 500: Internal Server Error
Error loading table from https://fbref.com/en/squads/91b092e1/2021/matchlogs/all_comps/gca/San-Diego-FC-Match-Logs-All-Competitions: HTTP Error 500: Internal Server Error
Error loading table from https://fbref.com/en/squads/bd97ac1f/2021/matchlogs/all_comps/shooting/St-Louis-City-Match-Logs-All-Competitions: HTTP Error 500: Internal Server Error
Error loading table from https://fbref.com/en/squads/bd97ac1f/2021/matchlogs/all_comps/gca/St-Louis-City-Match-Logs-All-Competiti

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
import numpy as np


def agg_relevant_stats(year, df):
    # Map points for all rows first
    df['Pts'] = df['Result_x'].map(POINTS_MAP)
    df['GF_x'] = df['GF_x'].apply(extract_number)

    # Set points to NaN or 0 where it's not Regular Season
    df.loc[df['Round_x'] != 'Regular Season', 'Pts'] = np.nan  # or `np.nan` if you want to ignore

    summary_df = df.groupby("Team")[LINEAR_COLS + ['Pts']].sum().reset_index()
    summary_df['Season'] = year
    summary_df = summary_df.rename(columns={
        "Result_x": "Result",
        "Round_x": "Round",
        "GF_x": "GF",
        "Sh_x": "SH"
    })
    return summary_df



In [ ]:
def get_relevant_stats(df):
    # Desired columns
    desired_cols = LINEAR_COLS + ['Team', 'Date', 'Season']
    # Only keep those that actually exist
    available_cols = [col for col in desired_cols if col in df.columns]

    # Now safely select
    df = df[available_cols]
    df= df.rename(columns={
            "Result_x": "Result",
            "Round_x": "Round",
            "GF_x": "GF",
            "Sh_x": "SH"
        })
    return df

In [ ]:
df_combined_list_agg_clean = []
df_combined_list = []
for year in RELEVANT_YEARS:
    if year == 2025:
        continue
    df = pd.read_csv(get_data_directory_matches(year))
    df["Season"] = year

    df_clean = get_relevant_stats(df.copy())
    df_combined_list.append(df_clean)

    df_agg_clean = agg_relevant_stats(year, df.copy())
    df_combined_list_agg_clean.append(df_agg_clean)


df_combined_clean = pd.concat(df_combined_list_agg_clean)
df_combined_clean.to_csv(DIRECTORY_COMBINED_MATCHES_CLEAN_AGG)

df_combined = pd.concat(df_combined_list)
df_combined.to_csv(DIRECTORY_COMBINED_MATCHES_CLEAN)
